In [ ]:
import ee
import geemap
from PIL import Image
from pathlib import Path

In [ ]:
# authenticate and initialize Earth Engine

ee.Authenticate()
ee.Initialize(project="gsapp-map")

dataset = ee.ImageCollection("GOOGLE/SATELLITE_EMBEDDING/V1/ANNUAL")

center = [-88.76878062783561, 44.29460729343232]
point = ee.Geometry.Point(-88.76878062783561, 44.29460729343232)
region = point.buffer(5000)
image2024 = dataset.filterDate("2024-01-01", "2025-01-01").filterBounds(region).mosaic()

In [ ]:
# download the images as tifs, iterating through the bands 3 at a time for RGBs

i = 0

while i + 2 < 64:
    band_1 = f"A{str(i).zfill(2)}"
    band_2 = f"A{str(i + 1).zfill(2)}"
    band_3 = f"A{str(i + 2).zfill(2)}"

    curr_img = image2024.select([band_1, band_2, band_3])

    region_trimmed = point.buffer(500).bounds()  # meters bound box
    scale = 1  # meters per pixel

    filename = f"data/wi/500m_resolution/tifs/{band_1}_{band_2}_{band_3}.tif"
    print(f"Exporting {filename}...")

    try:
        geemap.ee_export_image(
            curr_img,
            filename=filename,
            region=region_trimmed,
            scale=scale,
            verbose=True,
        )

    except Exception as e:
        print(f"Failed to export {filename}: {e}")

    i += 1

In [ ]:
# convert tifs to pngs

import rasterio
from rasterio.plot import reshape_as_image
import numpy as np

input_dir = Path("data/wi/500m_resolution/tifs")
output_dir = Path("data/wi/500m_resolution/pngs")

for tif_path in input_dir.glob("*.tif"):
    png_path = output_dir / tif_path.with_suffix(".png").name
    try:
        with rasterio.open(tif_path) as src:
            arr = src.read()  # shape: (bands, height, width)

            # Handle 1–3+ band cases
            if arr.shape[0] == 1:
                # single band: normalize to 0–255 grayscale
                band = arr[0]
                band = 255 * (band - band.min()) / (band.max() - band.min())
                rgb = np.stack([band] * 3, axis=-1)
            else:
                # take first 3 bands, normalize each
                arr = arr[:3]
                arr = np.clip(arr, np.nanpercentile(arr, 2), np.nanpercentile(arr, 98))
                arr = 255 * (arr - arr.min()) / (arr.max() - arr.min())
                rgb = reshape_as_image(arr)

            rgb = np.nan_to_num(rgb).astype(np.uint8)
            Image.fromarray(rgb).save(png_path)
            print(f"{tif_path.name} → {png_path.name}")

    except Exception as e:
        print(f"!!! Failed on {tif_path.name}: {e}")

In [ ]:
# Convert pngs to gif

# Path to your PNGs
png_dir = Path("data/wi/500m_resolution/pngs")
png_files = sorted(png_dir.glob("*.png"))

# Load images
images = [Image.open(p) for p in png_files]

# Save as GIF
gif_path = Path("data/wi") / "animation_500m_i1.gif"
images[0].save(
    gif_path,
    save_all=True,
    append_images=images[1:],
    duration=100,  # milliseconds per frame
    loop=0,  # infinite loop
)

print(f"Saved gif: {gif_path}")